<a href="https://colab.research.google.com/github/ParagBhokre/powerbi-enterprise-analytics/blob/main/powerbi_enterprise_data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Global random seed for reproducibility
np.random.seed(42)


In [2]:
date_range = pd.date_range(start="2019-01-01", end="2025-12-31")

date_dim = pd.DataFrame({
    "Date": date_range,
    "Year": date_range.year,
    "Month": date_range.month,
    "MonthName": date_range.month_name(),
    "YearMonth": date_range.strftime("%Y-%m"),
    "Quarter": "Q" + ((date_range.month - 1) // 3 + 1).astype(str),
    "IsWeekend": date_range.weekday >= 5
})

date_dim.to_excel("Date_Dim.xlsx", index=False)


In [3]:
geography_dim = pd.DataFrame({
    "GeographyID": range(1, 7),
    "Country": ["USA", "Germany", "India", "UK", "Australia", "Canada"],
    "Region": ["North America", "Europe", "Asia", "Europe", "APAC", "North America"],
    "Currency": ["USD", "EUR", "INR", "GBP", "AUD", "CAD"]
})

geography_dim.to_excel("Geography_Dim.xlsx", index=False)


In [4]:
num_products = 500

categories = {
    "Electronics": ["Phones", "Laptops", "Accessories"],
    "Apparel": ["Men", "Women", "Kids"],
    "Home": ["Furniture", "Kitchen", "Decor"]
}

product_rows = []
pid = 1

for cat, subs in categories.items():
    for sub in subs:
        for _ in range(num_products // 9):
            product_rows.append([
                pid,
                f"{cat[:3]}-{sub[:3]}-{pid}",
                cat,
                sub,
                np.random.choice(["BrandA", "BrandB", "BrandC"]),
                pd.to_datetime("2018-01-01") + pd.to_timedelta(np.random.randint(0, 1800), unit="D"),
                np.random.choice([True, False], p=[0.1, 0.9])
            ])
            pid += 1

product_dim = pd.DataFrame(product_rows, columns=[
    "ProductID", "ProductName", "Category", "SubCategory",
    "Brand", "LaunchDate", "DiscontinuedFlag"
])

product_dim.to_excel("Product_Dim.xlsx", index=False)


In [5]:
num_customers = 20000

customer_dim = pd.DataFrame({
    "CustomerID": range(1, num_customers + 1),
    "CustomerName": [f"Customer_{i}" for i in range(1, num_customers + 1)],
    "Gender": np.random.choice(["Male", "Female"], num_customers),
    "AgeGroup": np.random.choice(["18-25", "26-35", "36-50", "50+"], num_customers),
    "Segment": np.random.choice(["Retail", "Wholesale"], num_customers, p=[0.8, 0.2]),
    "SignupDate": pd.to_datetime("2018-01-01") + pd.to_timedelta(np.random.randint(0, 2200), unit="D"),
    "GeographyID": np.random.randint(1, 7, num_customers)
})

customer_dim.to_excel("Customer_Dim.xlsx", index=False)


In [6]:
num_stores = 200

store_dim = pd.DataFrame({
    "StoreID": range(1, num_stores + 1),
    "StoreName": [f"Store_{i}" for i in range(1, num_stores + 1)],
    "StoreType": np.random.choice(["Online", "Physical"], num_stores, p=[0.3, 0.7]),
    "GeographyID": np.random.randint(1, 7, num_stores),
    "OpenDate": pd.to_datetime("2015-01-01") + pd.to_timedelta(np.random.randint(0, 3000), unit="D"),
    "StoreSize": np.random.choice(["Small", "Medium", "Large"], num_stores)
})

store_dim.to_excel("Store_Dim.xlsx", index=False)


In [7]:
num_sales = 1000_000

sales_fact = pd.DataFrame({
    "SalesID": range(1, num_sales + 1),
    "OrderDate": np.random.choice(date_dim["Date"], num_sales),
    "ProductID": np.random.choice(product_dim["ProductID"], num_sales),
    "CustomerID": np.random.choice(customer_dim["CustomerID"], num_sales),
    "StoreID": np.random.choice(store_dim["StoreID"], num_sales),
    "Quantity": np.random.randint(1, 6, num_sales)
})

sales_fact["GrossRevenue"] = sales_fact["Quantity"] * np.random.uniform(20, 500, num_sales).round(2)
sales_fact["DiscountAmount"] = (sales_fact["GrossRevenue"] * np.random.uniform(0, 0.3, num_sales)).round(2)
sales_fact["NetRevenue"] = sales_fact["GrossRevenue"] - sales_fact["DiscountAmount"]
sales_fact["CostAmount"] = (sales_fact["NetRevenue"] * np.random.uniform(0.55, 0.75, num_sales)).round(2)

sales_fact.to_excel("Sales_Fact.xlsx", index=False)
